In [4]:
"""
README: City-Specific and Split-Specific YOLO Dataset Creator

This script:
- For each test split (e.g. 65-35, 75-25, 95-5), and for each city,
- Creates a new dataset directory containing:
    - symlinked 'train/' (full)
    - symlinked 'valid/' (full)
    - 'test/' with city-filtered symlinks for images/labels
    - an updated data.yaml for training/validation/testing on that split and city

How to use:
- Set 'original_dataset', 'split_base', and 'target_base' paths to match your directory structure.
- Update 'city_keywords' as needed.
- Run this script; each [split]-[city] subfolder will be created under 'target_base'.

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import os
import shutil
from pathlib import Path
import yaml

# Base dataset directory
original_dataset = Path('/PATH/TO/original-dataset-root')
train_dir = original_dataset / 'train'
valid_dir = original_dataset / 'valid'
data_yaml_path = original_dataset / 'data.yaml'

# City filter keywords
city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munich': ['munih'],
    'marseille': ['marsilya']
}

# Test split folders
split_base = original_dataset / 'splits'
splits = ['split_65-35', 'split_75-25', 'split_95-5']

# Target/output base
target_base = Path('/PATH/TO/output-dataset-root')

for split in splits:
    test_dir = split_base / split / 'test'

    for city, keywords in city_keywords.items():
        print(f"Creating dataset for {split} - {city.upper()}...")

        # Output directory for this split+city
        city_dataset_dir = target_base / f'{split}-{city}'

        # Remove if already exists
        if city_dataset_dir.exists():
            print(f"Removing old directory: {city_dataset_dir}")
            shutil.rmtree(city_dataset_dir)

        # Symlink train/
        dst_train = city_dataset_dir / 'train'
        dst_train.parent.mkdir(parents=True, exist_ok=True)
        if not dst_train.exists():
            os.symlink(train_dir, dst_train)
        print(f"Train folder symlinked.")

        # Symlink valid/
        dst_valid = city_dataset_dir / 'valid'
        if not dst_valid.exists():
            os.symlink(valid_dir, dst_valid)
        print(f"Valid folder symlinked.")

        # Filtered test/ symlinks
        city_test_images_dir = city_dataset_dir / 'test' / 'images'
        city_test_labels_dir = city_dataset_dir / 'test' / 'labels'
        city_test_images_dir.mkdir(parents=True, exist_ok=True)
        city_test_labels_dir.mkdir(parents=True, exist_ok=True)

        for img_path in (test_dir / 'images').glob('*.*'):
            img_name = img_path.name.lower()
            if any(keyword in img_name for keyword in keywords):
                dst_img = city_test_images_dir / img_path.name
                if not dst_img.exists():
                    os.symlink(img_path, dst_img)

                label_path = test_dir / 'labels' / (img_path.stem + '.txt')
                if label_path.exists():
                    dst_lbl = city_test_labels_dir / label_path.name
                    if not dst_lbl.exists():
                        os.symlink(label_path, dst_lbl)
                else:
                    print(f"Label not found (test): {label_path}")

        print(f"Test folder for {city} ({split}) ready.")

        # Create and update data.yaml
        with open(data_yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)

        data_yaml['train'] = str(dst_train / 'images')
        data_yaml['val'] = str(dst_valid / 'images')
        data_yaml['test'] = str(city_dataset_dir / 'test' / 'images')

        # Save YAML
        city_yaml_path = city_dataset_dir / 'data.yaml'
        with open(city_yaml_path, 'w') as f:
            yaml.dump(data_yaml, f)

        print(f"data.yaml for {city} ({split}) saved.\n")

print("All split + city datasets created successfully!")


🔵 split_65-35 - ISTANBUL için dataset oluşturuluyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü symlink yapıldı.
✅ test klasörü istanbul (split_65-35) için hazır.
✅ data.yaml istanbul (split_65-35) için kaydedildi.

🔵 split_65-35 - PARIS için dataset oluşturuluyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü symlink yapıldı.
✅ test klasörü paris (split_65-35) için hazır.
✅ data.yaml paris (split_65-35) için kaydedildi.

🔵 split_65-35 - MUNIH için dataset oluşturuluyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü symlink yapıldı.
✅ test klasörü munih (split_65-35) için hazır.
✅ data.yaml munih (split_65-35) için kaydedildi.

🔵 split_65-35 - MARSILYA için dataset oluşturuluyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü symlink yapıldı.
✅ test klasörü marsilya (split_65-35) için hazır.
✅ data.yaml marsilya (split_65-35) için kaydedildi.

🔵 split_75-25 - ISTANBUL için dataset oluşturuluyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü symlink yapıldı.
✅